In [1]:
# Starter code (run this first)
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps


In [2]:
def load_image(filepath):
    """Loads an image into a numpy array.
    Note: image will have 3 color channels [r, g, b]."""
    img = np.float32(Image.open(filepath))
    #img = ImageOps.grayscale(img)
    return cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
    #return (np.asarray(img).astype('uint8')/255)[:, :, :3]

In [3]:
sift = cv.SIFT_create() # sift detector uses difference of gaussian for feature detection

# get feature matches between two images using SIFT
def get_point_matches(img1, img2):
    """Returns matches as array: (feature track, image, coord)"""

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=10)
    search_params = dict(checks=100)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    good = []
    pts1 = []
    pts2 = []
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.85*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
        
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv.findFundamentalMat(pts1, pts2, cv.FM_LMEDS)
    # We select only inlier points
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    
    pt1 = np.append(pts1[0],1)
    pt2 = np.append(pts2[0],1)
    epi_cnst = np.transpose(pt2)@F@pt1
    print("Epipolar constraint is ", epi_cnst)
    
    f = 0.05
    K1 = [[f, 0, img1.shape[0]/2],
     [0, f, img1.shape[1]/2],
     [0, 0, 1]]
    
    K2 = [[f, 0, img1.shape[0]/2 + .00918],
     [0, f, img1.shape[1]/2 + .00246],
     [0, 0, 1]]
    
    
    E_mat = np.transpose(K2)@F@K1
    u, s, vh = np.linalg.svd(E_mat, full_matrices=False)
    newE = u@np.eye(3)@np.transpose(vh)
    u, s, vh = np.linalg.svd(newE, full_matrices=False)
    
    W = np.eye(3)
    
    R1 = u@W@np.transpose(vh)
    R2 = u@np.transpose(W)@np.transpose(vh)
    print("u is ", u)
    
    u = np.array(u)
    t1 = u[:,-1]
    t2 = -u[:,-1]
    
    
    return np.stack((pts1, pts2), axis=1), pts1, pts2, F, R1, R2, t1, t2
# combine feature matches
def combine_matches(matches_a, matches_b):
    """Assumes that the 0'th image is the same between them."""
    combined_matches = []
    for ii in range(matches_a.shape[0]):
        ma0 = matches_a[ii, 0]
        # Find the match in b
        mi = np.where((matches_b[:, 0] == ma0).all(axis=1))[0]

        # If a match is found, add to the array
        if mi.size > 0:
            ma = matches_a[ii]
            mb = matches_b[int(mi[0])]
            combined_matches.append(np.concatenate(
                (ma, mb[1:]), axis=0))

    return np.array(combined_matches)



In [4]:
def visualize_matches(img_a, img_b, matches, ax=None):
    #if ax is None:
        #fig = plt.figure(figsize=(20,20))
        #ax = plt.gca()
    
    sa = img_a.shape
    sb = img_b.shape
    sp = 40
    off = sa[1]+sp
    
    merged_imgs = np.zeros(
        (max(sa[0], sb[0]), sa[1]+sb[1]+sp),
        dtype=np.float)
    merged_imgs[0:sa[0], 0:sa[1]] = img_a
    merged_imgs[0:sb[0], sa[1]+sp:] = img_b
    #ax.imshow(merged_imgs)
    
    #for m in matches:
        #ax.plot([m[0][0], m[1][0]+off], [m[0][1], m[1][1]], 'r', alpha=0.5)

def get_match_colors(image_c, combined_matches):
    colors = []
    nm = combined_matches.shape[0]
    for mi in range(nm):
        m = combined_matches[mi, 0, :]
        colors.append(image_c[m[1]-1:m[1]+2,
                              m[0]-1:m[0]+2].sum(axis=0).sum(axis=0)/9)
    
    return colors

In [5]:
def rectify_two(test1,test2,pts1,pts2, F):
    png = "1.png"
    h1,w1 = test1.shape
    h2,w2 = test2.shape
    _, H1, H2 = cv.stereoRectifyUncalibrated(np.float32(pts1), np.float32(pts2), F, imgSize=(w1,h1))
    
    img1_rectified = cv.warpPerspective(test1, H1, (w1,h1))
    img2_rectified = cv.warpPerspective(test2, H2, (w2,h2))
    
    #fig, axes = plt.subplots(1,2, figsize=(15, 10))
    #axes[0].imshow(img1_rectified, cmap="gray")
    #axes[1].imshow(img2_rectified, cmap='gray')
    #axes[0].axhline(400)
    #axes[1].axhline(400)
    #axes[0].axhline(600)
    #axes[1].axhline(600)

    
    return img1_rectified, img2_rectified

In [6]:
def set_points(three_images_arr, i, depth_map):
    if len(three_images_arr) != 3:
        raise ValueError('Array must have 3 images')
    test1 = cv.imread(three_images_arr[0],0)
    test2 = cv.imread(three_images_arr[1],0)
    test3 = cv.imread(three_images_arr[2],0)
    matches1, pts1,pts2,F,R1,R2,t1,t2 = get_point_matches(test1, test2)
    matches2, pts1_1,pts2_1,F_1,R1_1,R2_1,t1_1,t2_1 = get_point_matches(test1, test3)
    
    visualize_matches(test1, test2, matches1, ax=None)
    visualize_matches(test1, test3, matches2, ax=None)
    
    img1_rectified, img2_rectified = rectify_two(test1,test2,pts1,pts2, F)
    img1_rectified, img3_rectified = rectify_two(test1,test3,pts1_1,pts2_1, F_1)
    
    combined_matches = combine_matches(np.array(matches1), np.array(matches2)) 
    colors = get_match_colors(load_image(three_images_arr[0]), combined_matches)

    return combined_matches, colors, img1_rectified, img2_rectified, img3_rectified, R1,R2,t1,t2,R1_1,R2_1,t1_1,t2_1 

In [7]:
test1 = load_image('./world_rotate/trans_01.png')
#fig1 = plt.figure()
#plt.imshow(test1[:, :, 0])

#fig2 = plt.figure()
test2 = load_image('./world_rotate/trans_02.png')
#plt.imshow(test2[:, :, 0])

#fig3 = plt.figure()
test3 = load_image('./world_rotate/trans_03.png')
#plt.imshow(test3[:, :, 0])

depth_map = cv.imread('./world_rotate/depth_map_world.png')
depth_map = cv.cvtColor(depth_map, cv.COLOR_BGR2GRAY)

#fig1 = plt.figure()
#plt.imshow(depth_map)

In [8]:
three_images =  ['./world_rotate/trans_01.png','./world_rotate/trans_02.png','./world_rotate/trans_03.png']
X, colors, img1_rectified, img2_rectified, img3_rectified, R1,R2,t1,t2, R1_1,R2_1,t1_1,t2_1 = set_points(three_images, 0, depth_map)

Epipolar constraint is  -0.006032048915757926
u is  [[-1.77992450e-01 -9.83968284e-01  1.11849973e-02]
 [-9.82082748e-01  1.78343474e-01  6.08858079e-02]
 [ 6.19044752e-02  1.47378715e-04  9.98082068e-01]]
Epipolar constraint is  -0.014964823868387267
u is  [[-9.97275062e-01 -2.98386356e-02  6.74692945e-02]
 [ 7.37729259e-02 -4.04263743e-01  9.11662427e-01]
 [ 7.26265829e-05  9.14155612e-01  4.05363433e-01]]


In [9]:
X[0:5]

array([[[829, 357],
        [809, 340],
        [801, 321]],

       [[869, 488],
        [820, 472],
        [785, 451]],

       [[882, 501],
        [831, 486],
        [792, 465]],

       [[887, 502],
        [834, 487],
        [794, 468]],

       [[890, 442],
        [841, 428],
        [804, 410]]])

In [10]:
def get_depth_2(img1_rectified, img2_rectified):
    stereoSGBM = cv.StereoSGBM_create(minDisparity=0, 
                                   numDisparities = 64,
                                   blockSize = 3,
                                   P1 = 8*1*3*3, 
                                   P2 = 32*1*3*3
                                  )
    dispSGBM = stereoSGBM.compute(img1_rectified, img2_rectified).astype(np.float32) / 16
    #fig1 = plt.figure()
    #plt.imshow(dispSGBM, 'gray')
    #plt.colorbar()
    
    stereoBM = cv.StereoBM_create(numDisparities=16, blockSize=15)
    dispBM = stereoBM.compute(img1_rectified, img2_rectified)
    
    #fig2 = plt.figure()
    #plt.imshow(dispBM, 'gray')
    #plt.colorbar()
dispSGBM_1 = get_depth_2(img1_rectified, img2_rectified)

dispSGBM_2 = get_depth_2(img2_rectified, img3_rectified)

In [11]:
depth_map = cv.imread('./world_rotate/depth_map_world.png',0)

In [68]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
N1 = test1.shape[0]
N2 = test1.shape[1]
N3 = test1.shape[0]
ma = np.zeros((N1,N2,N3))#

#xyz = np.random.normal(0, 1, (100, 3)).astype(np.float32)
#features = np.ones_like(xyz)
#oc_from_pcl = pyoctnet.Octree.create_from_pc(xyz, features, vx_res,vx_res,vx_res, normalize=True, n_threads=n_threads)



#fig = plt.figure()
#ax = fig.gca(projection='3d')
#ax.set_aspect('auto')
#ax.voxels(ma, edgecolor="k")

#plt.show()

MemoryError: Unable to allocate 16.7 GiB for an array with shape (1080, 1920, 1080) and data type float64

In [16]:
pa = [np.array(i) for i in X[:,0].tolist()]
pb = [np.array(i) for i in X[:,1].tolist()]
pc = [np.array(i) for i in X[:,2].tolist()]

In [66]:
f = 188.9 

# Camera matrix
K = [[f, 0, img1_rectified.shape[0]/2],
     [0, f, img1_rectified.shape[1]/2],
     [0, 0, 1]]
R = np.eye(3)
ta = [[0],[0],[1]]
Pb = K @ np.concatenate((R, ta), axis=1)

def rot_trans(Pa):
    R = [[np.cos(15),-np.sin(15),0],
         [np.sin(15),np.cos(15),0],
         [0,0,1]]     
    tb = [[34.720032502], [9.3071030883], [0]] 
    Pb = Pa @ np.concatenate((R, tb), axis=1)
    return Pb

def get_projected_point(P, X):
    Xh = np.concatenate((X, [1]), axis=0)
    PX_inh = P @ Xh
    PX = PX_inh[:2]/PX_inh[2]
    return PX

X0 = np.array([ 0.85244616, 0.9508618, -0.51819406])
points_3D = [X0]


i_coords, j_coords, k_coords = np.meshgrid(range(N1), range(N2), range(N3), indexing='ij')
coordinate_grid = np.array([i_coords, j_coords,k_coords])

print(coordinate_grid[:, 0, 0, 0])

import os

filelist = os.listdir('./world_rotate/')
filelist = sorted(filelist,key=lambda x: int(os.path.splitext(x)[0]))

for i in range(len(filelist)-1):
    test1 = filelist[i]
    test2 = filelist[i+1]
    Pa = Pb
    Pb = rot_trans(Pa)
    for i in range(N1):
        for j in range(N2):
            for k in range(N3):
                pt = coordinate_grid[:, i, j, k]
                TD_X = get_projected_point(Pa,pt)
                TD_Y = get_projected_point(Pb,pt)
                #need to come up with a better metric
                #and perhaps down 
                if((TD_X[0] < test1.shape[0] and TD_Y[0] < test1.shape[0]) and (TD_X[1] < test1.shape[1] and TD_Y1[1] < test.shape[1]) or
                   and (test1[TD_X[0],TD_X[1]] - test2[TD_Y[0],TD_Y[1]] == 0) and test1[TD_X[0],TD_X[1]] != 255):
                    ma[i,j,k] = 1


def visualize_projected_points(image, P, points_3D, verbose=False):
    plt.figure(dpi=100)
    plt.imshow(image)
    for X in points_3D:
        x = get_projected_point(P, X)
        if verbose:
            print(x)
        plt.plot(x[0], x[1], 'ko')

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect('auto')
ax.voxels(ma, edgecolor="k")
plt.show()

SyntaxError: invalid syntax (<ipython-input-66-de7167cd2d24>, line 53)

In [65]:

K = [[f, 0, img1_rectified.shape[0]/2, 0],
 [0, f, img1_rectified.shape[1]/2, 0],
 [0, 0, 1, 0]]

R_inv = np.asmatrix([[np.cos(15),np.sin(15),0],
     [-np.sin(15),np.cos(15),0],
     [0,0,0]])  

tb = np.asmatrix([[34.720032502], [9.3071030883], [1]])

R_inv_t = R_inv @ tb
RRT = np.concatenate((R_inv, R_inv_t), axis=1)


p_11 = RRT[0,0]
p_12 = RRT[0,1]
p_13 = RRT[0,2]
p_14 = RRT[0,3]

p_21 = RRT[1,0]
p_22 = RRT[1,1]
p_23 = RRT[1,2]
p_24 = RRT[1,3]

p_31 = RRT[2,0]
p_32 = RRT[2,1]
p_33 = RRT[2,2]
p_34 = RRT[2,3]


def plane_1(pt):
    X, Y, Z = pt[:]
    
X*(p_11 - u*p_31) + Y*(p_12 - u*P_32) + Z*(p_13 - u*p_33) + p_14 - u*p_34 = 0
#X*(p_21 - v*p_31) + Y*(p_22 - v*P_32) + Z*(p_23 - v*p_33) + p_24 - v*p_34 = 0

[[-0.75968791  0.65028784  0.        ]
 [-0.65028784 -0.75968791  0.        ]
 [ 0.          0.          0.        ]]
[[-20.32409306]
 [-29.64850867]
 [  0.        ]]
[[ -0.75968791   0.65028784   0.         -20.32409306]
 [ -0.65028784  -0.75968791   0.         -29.64850867]
 [  0.           0.           0.           0.        ]]


In [54]:
import bpy
import numpy as np

import bpy

filepath = "C:/Users/Helena Gray/Downloads/world_files/WorldBlender.blend"

# load a single scene we know the name of.
with bpy.data.libraries.load(filepath) as (data_from, data_to):
    data_to.scenes = ["Scene"]
    
    
def get_calibration_matrix_K_from_blender(mode='simple'):

    scene = bpy.context.scene

    scale = scene.render.resolution_percentage / 100
    width = scene.render.resolution_x * scale # px
    height = scene.render.resolution_y * scale # px

    camdata = scene.camera.data

    if mode == 'simple':

        aspect_ratio = width / height
        K = np.zeros((3,3), dtype=np.float32)
        K[0][0] = width / 2 / np.tan(camdata.angle / 2)
        K[1][1] = height / 2. / np.tan(camdata.angle / 2) * aspect_ratio
        K[0][2] = width / 2.
        K[1][2] = height / 2.
        K[2][2] = 1.
        K.transpose()
    
    if mode == 'complete':

        focal = camdata.lens # mm
        sensor_width = camdata.sensor_width # mm
        sensor_height = camdata.sensor_height # mm
        pixel_aspect_ratio = scene.render.pixel_aspect_x / scene.render.pixel_aspect_y

        if (camdata.sensor_fit == 'VERTICAL'):
            # the sensor height is fixed (sensor fit is horizontal), 
            # the sensor width is effectively changed with the pixel aspect ratio
            s_u = width / sensor_width / pixel_aspect_ratio 
            s_v = height / sensor_height
        else: # 'HORIZONTAL' and 'AUTO'
            # the sensor width is fixed (sensor fit is horizontal), 
            # the sensor height is effectively changed with the pixel aspect ratio
            pixel_aspect_ratio = scene.render.pixel_aspect_x / scene.render.pixel_aspect_y
            s_u = width / sensor_width
            s_v = height * pixel_aspect_ratio / sensor_height

        # parameters of intrinsic calibration matrix K
        alpha_u = focal * s_u
        alpha_v = focal * s_v
        u_0 = width / 2
        v_0 = height / 2
        skew = 0 # only use rectangular pixels

        K = np.array([
            [alpha_u,    skew, u_0],
            [      0, alpha_v, v_0],
            [      0,       0,   1]
        ], dtype=np.float32)
    
    return K

ModuleNotFoundError: No module named 'bpy'